In [1]:
import pandas as pd

addrs = pd.read_csv('addr_info.csv', index_col=0)
addrs = addrs[~((addrs.duplicated(subset=['address'])) & 
          (~addrs['name'].isin(['binance', 'coinbase', 'bullish'])))].drop_duplicates(subset=['address'])
addrs.name = addrs.name.str.lower()
feats = ['name', 'serviceType', 'sub1Service', 'walletPurpose', 'sub1Wallet']
addrs.columns
# addrs[addrs.address.isin(addrs[addrs.duplicated(subset=['address'])].address)]

Index(['address', 'name', 'isContract', 'isWhale', 'network', 'serviceType',
       'sub1Service', 'walletPurpose', 'sub1Wallet'],
      dtype='object')

In [2]:
import duckdb

# Initialize DuckDB connection
try:
    conn = duckdb.connect('duckdb_test.db', read_only=False)
    print('DuckDB connection established successfully')
except Exception as e:
    print(f'Error connecting to DuckDB: {str(e)}')
    exit(1)


DuckDB connection established successfully


In [4]:

# Create DuckLake metadata
try:
    conn.execute("""
        CREATE OR REPLACE TABLE metadata (
            table_name VARCHAR,
            data_path VARCHAR,
            partition_columns VARCHAR,
            data_format VARCHAR
        )
    """)
    
    conn.execute("""
        INSERT INTO metadata
        VALUES (
            'ethereum_token_transfer',
            'F:\token_transfers\ethereum_token_transfer',
            'year,month',
            'parquet'
        )
    """)
    
    print('Metadata table created successfully')
    
    # Show metadata
    metadata = conn.execute("SELECT * FROM metadata").df()
    print('\nMetadata:')
    print(metadata)
    
except Exception as e:
    print(f'Error creating metadata: {str(e)}')
    exit(1)
`

Metadata table created successfully

Metadata:
                table_name                                   data_path  \
0  ethereum_token_transfer  F:\token_transfers\ethereum_token_transfer   

  partition_columns data_format  
0        year,month     parquet  


In [5]:
%pwd

'f:\\token_transfers'

In [6]:



tokens = dict(
    tether='0xdAC17F958D2ee523a2206206994597C13D831ec7',
    usdc='0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
    dai='0x6B175474E89094C44Da98b954EedeAC495271d0F',
    fdusd='0xc5f0f7b66764F6ec8C8Dff7BA683102295E16409',
    weth='0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
    wbtc='0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599',
    bnb='0xB8c77482e45F1F44dE1745F52C74426C631bDD52',
    ada='0x3ee2200efb3400fabb9aacf31297cbdd1d435d47',
    link='0x514910771af9ca656af840dff83e8264ecf986ca'
)


In [ ]:
for year in range(2021,2025):
    for month in range(1, 13):
        # Create the table
        df_martial_law = conn.execute(f"""
            SELECT *
            FROM read_parquet('ethereum_token_transfer/*/*/*.parquet', hive_partitioning = true)
            WHERE year={year} AND month={month}
        """).df()

        for k, v in tokens.items():
            df_martial_law.loc[df_martial_law.token_address==v.lower(), 'token_name'] = k
            
        for v in ['token', 'from', 'to']:
            df_martial_law = df_martial_law.merge(
            addrs[['address']+feats], 
            left_on=f'{v}_address', right_on='address', 
            how="left", validate='m:1'
        ).rename(columns={c:f'{v}_{c}' for c in feats}).drop(columns=['address'])
        df_martial_law['token_name'] = df_martial_law.filter(like='token_name').bfill(axis=1).iloc[:, 0]
        df_martial_law = df_martial_law.loc[:, ~df_martial_law.columns.duplicated()]



        df_ml_graph = df_martial_law[[
            'from_name', 'to_name', 'transaction_hash', 'raw_amount', 'block_timestamp', 'from_address', 'to_address'
        ]].copy().reset_index(drop=True)
        for v in ['from', 'to']:
            df_ml_graph.loc[df_ml_graph[f'{v}_name'].isna(), f'{v}_name'] =  df_ml_graph.loc[df_ml_graph[f'{v}_name'].isna(), f'{v}_address' ]
        

        df_ml_graph = df_ml_graph.drop(columns=['from_address', 'to_address'])
        sims = pd.concat([
            df_ml_graph.from_name.value_counts().reset_index().rename(columns={'from_name': 'name'}),
            df_ml_graph.to_name.value_counts().reset_index().rename(columns={'to_name': 'name'})])
        simss = sims.groupby('name').sum('count')
        simss['year'] = year
        simss['month'] = month
        simss.to_csv(f'meta/{year}_{month}.csv')

In [ ]:
# import networkx as nx

# G = nx.from_pandas_edgelist(df_ml_graph, source='from_name', target='to_name', edge_attr='raw_amount')
# clusters = nx.community.greedy_modularity_communities(G)

# # 클러스터 라벨 부여
# cluster_map = {}
# for i, cluster in enumerate(clusters):
#     for addr in cluster:
#         cluster_map[addr] = f'cluster_{i}'

In [ ]:
# df_ml_graph['from_cluster'] = df_ml_graph['from_name'].map(cluster_map)
# df_ml_graph['to_cluster'] = df_ml_graph['to_name'].map(cluster_map)

In [116]:
df_martial_law_compact = df_martial_law[
    ['block_timestamp', 'raw_amount', 'transaction_hash'] + [
        f'{v}_{c}' for v in ['from', 'to', 'token'] for c in feats
    ]
].sort_values(by='block_timestamp').reset_index(drop=True).copy()



df_martial_law_compact = df_martial_law_compact.fillna('undefined')
df_martial_law_compact

InvalidIndexError: (0            True
1            True
2           False
3           False
4            True
            ...  
25356870     True
25356871     True
25356872    False
25356873     True
25356874    False
Name: from_name, Length: 25356875, dtype: bool, 'from_address')

In [108]:
df_martial_law_compact.columns

Index(['block_timestamp', 'raw_amount', 'transaction_hash', 'from_name',
       'from_serviceType', 'from_sub1Service', 'from_walletPurpose',
       'from_sub1Wallet', 'to_name', 'to_serviceType', 'to_sub1Service',
       'to_walletPurpose', 'to_sub1Wallet', 'token_name', 'token_serviceType',
       'token_sub1Service', 'token_walletPurpose', 'token_sub1Wallet'],
      dtype='object')

In [ ]:
df_martial_law_compact[(df_martial_law_compact.token_name=='tether') & (df_martial_law_compact.transaction_hash=='0xbc30eb2b2d2a0c39e8c7b05f24ab5f2944aaf4955ef20bacbab2310cc17b6f26')] #.transaction_hash.at[25356839]

,block_timestamp,raw_amount,transaction_hash,from_name,from_serviceType,from_sub1Service,from_walletPurpose,from_sub1Wallet,to_name,to_serviceType,to_sub1Service,to_walletPurpose,to_sub1Wallet,token_name,token_serviceType,token_sub1Service,token_walletPurpose,token_sub1Wallet
25356800,2024-12-31 23:59:59,28070,0xbc30eb2b2d2a0c39e8c7b05f24ab5f2944aaf4955ef2...,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,tether,undefined,undefined,undefined,undefined
25356812,2024-12-31 23:59:59,93567961,0xbc30eb2b2d2a0c39e8c7b05f24ab5f2944aaf4955ef2...,1inch,DeFi,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,tether,undefined,undefined,undefined,undefined
25356829,2024-12-31 23:59:59,93539891,0xbc30eb2b2d2a0c39e8c7b05f24ab5f2944aaf4955ef2...,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,tether,undefined,undefined,undefined,undefined
25356833,2024-12-31 23:59:59,93567961,0xbc30eb2b2d2a0c39e8c7b05f24ab5f2944aaf4955ef2...,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,undefined,tether,undefined,undefined,undefined,undefined
25356839,2024-12-31 23:59:59,93567961,0xbc30eb2b2d2a0c39e8c7b05f24ab5f2944aaf4955ef2...,undefined,undefined,undefined,undefined,undefined,1inch,DeFi,undefined,undefined,undefined,tether,undefined,undefined,undefined,undefined
